## Описание бизнеса:
    Клининговая компания 

## Задачи анализа:
1. Посчитать показатели Retention Rate седьмого дня и ARPPU за два периода: до(май 2019) и после внедрения фичи(июнь 2019).
2. Сформировать выводы по анализу.

- Рассчитаем retention 7 дня до релиза и полсе 
- Рассчитаем ARPPU до релиза и полсе
- Сделаем выводы 

Импортируем необходимые библиотеки

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import psycopg2
import psycopg2.extras 
import warnings
warnings.filterwarnings('ignore')

## 1. Рассчитаем retention 7 дня до релиза и полсе

Создадим датафрейм из зарегистрированных пользователей за весь период 

In [2]:
def getRegisteredUsersCount(): #выполняем SQL-запрос через функцию 
    query = '''SELECT r.registration_date, count(distinct(r.user_id)) as registered_users_count
    FROM case9.registration r
    WHERE r.registration_date >= '2019-05-01' and r.registration_date < '2019-07-01'
    GROUP BY 1
    '''.format() #пользователи, которые зарегистировались за май и июнь 
    conn = psycopg2.connect("dbname='skillfactory' user='skillfactory' host='84.201.134.129' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data

registered_users = pd.DataFrame(getRegisteredUsersCount())  #позволяет выполнить функцию запроса, а затем записать полученные данные в датафрейм

In [3]:
registered_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61 entries, 0 to 60
Data columns (total 2 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   registration_date       61 non-null     object
 1   registered_users_count  61 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 1.1+ KB


Меняем тип object на datetime

In [4]:
registered_users['registration_date'] = pd.to_datetime(registered_users['registration_date']) 

Создадим датафрейм из активных пользователей за весь период 

In [5]:
def getActiveUsersCountWithCohorts():
    query = '''SELECT ac.activity_date, r.registration_date, count(distinct(ac.user_id)) as active_users_count
    FROM case9.user_activity ac
    LEFT JOIN case9.registration r ON r.user_id = ac.user_id
    WHERE r.registration_date >= '2019-05-01' and r.registration_date < '2019-07-01'
    GROUP BY 1,2
    '''.format()
    conn = psycopg2.connect("dbname='skillfactory' user='skillfactory' host='84.201.134.129' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data

active_users = pd.DataFrame(getActiveUsersCountWithCohorts())

In [6]:
active_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1891 entries, 0 to 1890
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   activity_date       1891 non-null   object
 1   registration_date   1891 non-null   object
 2   active_users_count  1891 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 44.4+ KB


Меняем тип object на datetime

In [7]:
for col in ['activity_date','registration_date']:
    active_users[col] = pd.to_datetime(active_users[col])

Объединим таблицы в единый датафрейм и рассчитаем lifetime

In [8]:
merged_table = active_users.merge(registered_users,on=['registration_date'],how='left')
merged_table['lifetime'] = merged_table['activity_date'] - merged_table['registration_date']
merged_table['lifetime'] = merged_table['lifetime']/np.timedelta64(1,'D') #разделим на константу, для получения целого числа 
merged_table['lifetime'] = merged_table['lifetime'].astype(int) # приведем тип к целому числу
merged_table.head()

,activity_date,registration_date,active_users_count,registered_users_count,lifetime
0,2019-05-01,2019-05-01,2590,6516,0
1,2019-05-02,2019-05-01,2146,6516,1
2,2019-05-02,2019-05-02,2049,5309,0
3,2019-05-03,2019-05-01,1637,6516,2
4,2019-05-03,2019-05-02,1817,5309,1


Создадим таблицу с данными о доходах за весь период 

In [9]:
def getRevenue():
    query = '''SELECT re.date, r.registration_date, count(distinct(re.user_id)) as users_count_with_revenue, sum(re.revenue) as revenue
    FROM case9.user_revenue re
    LEFT JOIN case9.registration r ON r.user_id = re.user_id
    WHERE r.registration_date >= '2019-05-01' and r.registration_date < '2019-07-01'
    GROUP BY 1,2
    '''.format()
    conn = psycopg2.connect("dbname='skillfactory' user='skillfactory' host='84.201.134.129' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data

revenue = pd.DataFrame(getRevenue())

In [10]:
revenue.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1745 entries, 0 to 1744
Data columns (total 4 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   date                      1745 non-null   object 
 1   registration_date         1745 non-null   object 
 2   users_count_with_revenue  1745 non-null   int64  
 3   revenue                   1745 non-null   float64
dtypes: float64(1), int64(1), object(2)
memory usage: 54.7+ KB


Меняем тип object на datetime

In [11]:
for col in ['date','registration_date']:
    revenue[col] = pd.to_datetime(revenue[col])

Объединим данные по доходам с получившимся датафреймом

In [12]:
revenue = revenue.rename(columns={'date':'activity_date'}) #переименнуем столбец для объединения
merged_revenue = merged_table.merge(revenue,on=['registration_date','activity_date'],how='left')
merged_revenue.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1891 entries, 0 to 1890
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   activity_date             1891 non-null   datetime64[ns]
 1   registration_date         1891 non-null   datetime64[ns]
 2   active_users_count        1891 non-null   int64         
 3   registered_users_count    1891 non-null   int64         
 4   lifetime                  1891 non-null   int32         
 5   users_count_with_revenue  1745 non-null   float64       
 6   revenue                   1745 non-null   float64       
dtypes: datetime64[ns](2), float64(2), int32(1), int64(2)
memory usage: 110.8 KB


Заменим пустые значени 

In [13]:
for col in ['revenue','users_count_with_revenue']:
    merged_revenue[col] = merged_revenue[col].fillna(0)
merged_revenue['users_count_with_revenue'] = merged_revenue['users_count_with_revenue'].astype(int)
merged_revenue.head()

,activity_date,registration_date,active_users_count,registered_users_count,lifetime,users_count_with_revenue,revenue
0,2019-05-01,2019-05-01,2590,6516,0,1672,2357.590
1,2019-05-02,2019-05-01,2146,6516,1,1420,514.679
2,2019-05-02,2019-05-02,2049,5309,0,1367,1629.140
3,2019-05-03,2019-05-01,1637,6516,2,1048,390.952
4,2019-05-03,2019-05-02,1817,5309,1,1164,533.608


Выделим данные по retention 7 для пользователей привлеченных до релиза (май 2019)

In [14]:
may_7 = merged_revenue[merged_revenue['lifetime']==7]\
[merged_revenue['registration_date']>="2019-05-01"][merged_revenue['registration_date']<"2019-06-01"]
may_7.head()

,activity_date,registration_date,active_users_count,registered_users_count,lifetime,users_count_with_revenue,revenue
28,2019-05-08,2019-05-01,953,6516,7,616,63.9065
37,2019-05-09,2019-05-02,745,5309,7,490,313.7220
47,2019-05-10,2019-05-03,895,6266,7,571,110.7260
58,2019-05-11,2019-05-04,1186,8639,7,750,124.8590
70,2019-05-12,2019-05-05,1358,9787,7,885,174.9240


Рассчитаем retention 7 дня (май 2019)

In [15]:
retention_may = (may_7['active_users_count'].sum() / may_7['registered_users_count'].sum())*100
print( 'Retention 7 дня для пользователей, привлеченных до релиза : {:.2f}%'.format(retention_may))

Retention 7 дня для пользователей, привлеченных до релиза : 12.93%


Выделим данные по retention 7 для пользователей привлеченных после релиза (июнь 2019)

In [16]:
june_7 = merged_revenue[merged_revenue['lifetime']==7]\
[merged_revenue['registration_date']>="2019-06-01"][merged_revenue['registration_date']<"2019-07-01"]
june_7.head()

,activity_date,registration_date,active_users_count,registered_users_count,lifetime,users_count_with_revenue,revenue
736,2019-06-08,2019-06-01,545,4833,7,365,176.3160
767,2019-06-09,2019-06-02,683,5255,7,461,193.8390
798,2019-06-10,2019-06-03,484,4193,7,320,27.7548
829,2019-06-11,2019-06-04,511,4194,7,320,54.9610
860,2019-06-12,2019-06-05,472,3998,7,301,63.1554


Рассчитаем retention 7 дня (июнь 2019)

In [17]:
retention_june = (june_7 ['active_users_count'].sum() / june_7 ['registered_users_count'].sum())*100
print( 'Retention 7 дня для пользователей, привлеченных после релиза : {:.2f}%'.format(retention_june))

Retention 7 дня для пользователей, привлеченных после релиза : 12.56%


- Retention 7 дня в мае составляет 12.93%.
- Retention 7 дня в июне составляет 12,56%. 

Разница минимальна, но стоит выделить, что после внедрения фичи retention стал ниже 

## 2. Рассчитаем ARPPU до релиза и полсе

Выделим revenue за май и поделим на кол-во платящих майских пользователей

In [18]:
ar_may = merged_revenue['revenue']\
[merged_revenue['registration_date']>="2019-05-01"][merged_revenue['registration_date']<"2019-06-01"].sum()\
/merged_revenue['users_count_with_revenue']\
[merged_revenue['registration_date']>="2019-05-01"][merged_revenue['registration_date']<"2019-06-01"].sum()
print('До релиза ARPPU составил: {:.1f}'.format(ar_may))

До релиза ARPPU составил: 0.3


Выделим revenue за июнь и поделим на кол-во платящих июньских пользователей

In [19]:
ar_june = merged_revenue['revenue']\
[merged_revenue['registration_date']>="2019-06-01"][merged_revenue['registration_date']<"2019-07-01"].sum()\
/ merged_revenue['users_count_with_revenue']\
[merged_revenue['registration_date']>="2019-06-01"][merged_revenue['registration_date']<"2019-07-01"].sum()
print('После релиза ARPPU составил: {:.1f}'.format(ar_june))

После релиза ARPPU составил: 0.4


## Выводы:
1. Retention 7 дня почти не изменился, а именно в мае до внедрения фичи 12.93%, в июне посде внедерния 12.56%
2. ARPPU изменился более существенно с 0.3 до 0.4, что является приростом на 30%.

Можно сделать вывод, что на данный момент фича является эффективной. 